# Лабораторная работа №1. Сбор новостного датасета
## Выполнила: Залесская Галина, 16ПМИ

In [1]:
import os
import numpy as np
import lxml
from lxml import objectify, etree, html
import json
import urllib3
from io import StringIO, BytesIO
import newspaper
from newspaper import Article
import pandas as pd

In [2]:
categories = ["world", "health", "technology", "science_and_environment", "uk", 
              "business", "entertainment_and_arts", "sport"]
root_URL = 'https://www.bbc.com'

In [3]:
import certifi
def get_urls_from_page(URL):
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    r = http.request('GET', URL)
    parser = etree.HTMLParser()
    page = etree.parse(BytesIO(r.data), parser)
    
    article_links = []
    #the main article
    #Очень страшные и разные теги
    article_links += (page.xpath(".//div/a[@class = 'gs-c-promo-heading gs-o-faux-block-link__overlay-link gel-paragon-bold gs-u-mt+ nw-o-link-split__anchor']/@href"))
    article_links += (page.xpath(".//div/a[@class = 'gs-c-promo-heading gs-o-faux-block-link__overlay-link gel-paragon-bold nw-o-link-split__anchor']/@href"))
    #extra articles
    article_links += (page.xpath(".//span/a[@class='gel-layout__item nw-o-link-split__anchor gs-u-pt- gs-u-pb- gs-u-display-block']/@href"))
    article_links += (page.xpath(".//div/a[@class='gs-c-promo-heading gs-o-faux-block-link__overlay-link sp-o-link-split__anchor gel-double-pica-bold']/@href"))
    article_links += (page.xpath(".//div/a[@class='gs-c-promo-heading gs-o-faux-block-link__overlay-link sp-o-link-split__anchor gel-pica-bold']/@href"))
    
    article_links += (page.xpath(".//span/a[@class='nw-o-link-split__anchor gs-u-pt- gs-u-pb- gs-u-display-block nw-o-link-split__text']/@href"))
    
    article_links += page.xpath(".//div/a[@class = 'gs-c-promo-heading gs-o-faux-block-link__overlay-link gel-pica-bold nw-o-link-split__anchor']/@href")
    article_links = [l for l in article_links if l.startswith('/')] #clean unrelated to news (articles, stories)
    
    print('from URL {} got {} articles'.format(URL, len(article_links)))
    return article_links

In [4]:
#мало статей на главной странице, поэтому приходится искать ссылки на уже взятых статьях
def get_extra_urls_from_article_page(URL):
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    r = http.request('GET', URL)
    parser = etree.HTMLParser()
    page = etree.parse(BytesIO(r.data), parser)
    
    article_links = []
    #the extra articles
    article_links += (page.xpath(".//a[@class='unit__link-wrapper']/@href"))
    article_links = [l for l in article_links if l.startswith('/')] #clean unrelated to news (articles, stories)
    
    if len(article_links) > 0:
        print('from URL {} additionally got {} articles'.format(URL, len(article_links)))
    return article_links

In [5]:
def get_urls_from_main_page(root_URL):
    articles_links = []
    news = '/news'
    for cat in [''] + categories:
        if cat == "sport":
            news = ""
        URL = root_URL + news + '/' + cat
        #cпортивные статьи начинаются с /sport, а все остальные с /news/category_name

        cat_articles = get_urls_from_page(URL)
        articles_links += cat_articles

        if cat in ["health", "world", "technology", "science_and_environment", "entertainment_and_arts"]:
            for article in cat_articles:
                url = root_URL + article
                articles_links += get_extra_urls_from_article_page(url)
    return list(set(articles_links))

In [6]:
articles_links = get_urls_from_main_page(root_URL)

from URL https://www.bbc.com/news/ got 41 articles
from URL https://www.bbc.com/news/world got 56 articles
from URL https://www.bbc.com/news/world-asia-51689443 additionally got 6 articles
from URL https://www.bbc.com/news/world-asia-51689443 additionally got 6 articles
from URL https://www.bbc.com/news/world-asia-51689443 additionally got 6 articles
from URL https://www.bbc.com/news/world-asia-49192495 additionally got 9 articles
from URL https://www.bbc.com/news/world-asia-47733079 additionally got 9 articles
from URL https://www.bbc.com/news/world-asia-49192495 additionally got 9 articles
from URL https://www.bbc.com/news/world-asia-47733079 additionally got 9 articles
from URL https://www.bbc.com/news/world-asia-49192495 additionally got 9 articles
from URL https://www.bbc.com/news/world-asia-47733079 additionally got 9 articles
from URL https://www.bbc.com/news/world-asia-49192495 additionally got 9 articles
from URL https://www.bbc.com/news/world-asia-47733079 additionally got 9 

from URL https://www.bbc.com/news/business-51658693 additionally got 1 articles
from URL https://www.bbc.com/news/world-asia-china-51658145 additionally got 3 articles
from URL https://www.bbc.com/news/science-environment-24021772 additionally got 9 articles
from URL https://www.bbc.com/news/science-environment-51590080 additionally got 3 articles
from URL https://www.bbc.com/news/world-europe-49918719 additionally got 4 articles
from URL https://www.bbc.com/news/uk got 55 articles
from URL https://www.bbc.com/news/business got 57 articles
from URL https://www.bbc.com/news/entertainment_and_arts got 36 articles
from URL https://www.bbc.com/news/world-europe-51684494 additionally got 7 articles
from URL https://www.bbc.com/news/world-europe-51684494 additionally got 7 articles
from URL https://www.bbc.com/news/world-europe-51684494 additionally got 7 articles
from URL https://www.bbc.com/news/world-europe-51494929 additionally got 4 articles
from URL https://www.bbc.com/news/world-europ

In [7]:
def set_category(articles): #берем категорию из названия ссылки, отбрасывая попавшие случайно
    articles_with_category = []
    for art in articles_links:
        cat = None
        if art.startswith('/sport'):
            cat = "sport"
        else:
            for c in categories:
                if art.startswith('/news/'+c):
                    cat = c
                    break
        if cat:
            articles_with_category.append([cat, art])
    return articles_with_category


In [8]:
print(str(len(articles_links)), ' статей до вычленения нужных категорий')
articles_with_category = set_category(articles_links)
print(str(len(articles_with_category)), ' после вычленения нужных категорий')
#получаем 249 статей

360  статей до вычленения нужных категорий
277  после вычленения нужных категорий


In [9]:
def parse_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.title, article.text, list(article.tags) + get_tags(url)

In [10]:
def get_tags(url): #стандартное тегирование работает плохо, поэтому улучшим его, подглядев в Related topics
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    r = http.request('GET', url)
    parser = etree.HTMLParser()
    page = etree.parse(BytesIO(r.data), parser)

    return page.xpath(".//ul/li[@class = 'tags-list__tags']/a/text()")

In [11]:
import xml.etree.ElementTree as xml

def create_xml(news):

  root = xml.Element("catalog")

  for [category_, url] in news:
        
    print('.', end = '')
    title_, text_, tags_ = parse_article(root_URL + url)
    if text_ == '':
        print("Empty text")
        continue 

#     print(title, content, tags)
    appt = xml.SubElement(root, "news_id")
    appt.text = root_URL + url

    category = xml.SubElement(appt, "category")
    category.text = category_

    title = xml.SubElement(appt, "title")
    title.text = title_

    text = xml.SubElement(appt, "text")
    text.text = text_

    tags = xml.SubElement(appt, "tags")
    tags.text = ', '.join(tags_)

  tree = xml.ElementTree(root)

  with open("output2.xml", "wb") as fh:
      tree.write(fh)

In [ ]:
create_xml(articles_with_category)

............................................................................................................................................................................................................................